In [ ]:
# 1. MSA coefficients by land use type and intensity (from True Price table)
msa_coefficient_by_land_use = {
    "natural_forest_minimal": 0.15,
    "natural_forest_light": 0.3,
    "natural_forest_intense": 0.5,
    "plantation_forest_minimal": 0.7,
    "plantation_forest_light": 0.75,
    "plantation_forest_intense": 0.8,
    "cropland_minimal": 0.6,
    "cropland_light": 0.7,
    "cropland_intense": 0.9,
    "pasture_light": 0.2,
    "pasture_moderate": 0.4,
    "pasture_heavy": 0.7,
    "urban": 0.95
}

# 2. Biome ESS values (€ per MSA·ha·year)
biome_ess_values = {
    'grassland': 2427,
    'temperate_forest': 1014,
    'wetland_inland': 14871,
    'wetland_coastal': 10939,
    'woodlands': 1369
}

# 3. Biome shares for the Netherlands
biome_share = {
    'grassland': 0.68,
    'temperate_forest': 0.14,
    'wetland_inland': 0.08,
    'wetland_coastal': 0.4,
    'woodlands': 0.6
}

# 4. Restoration costs (€ per MSA·ha), from True Price Table 4
restoration_costs = {
    'grassland': 271,
    'temperate_forest': 3595,
    'wetland_inland': 15930,
    'wetland_coastal': 6355,
    'woodlands': 1369
}

In [ ]:
# 5. Function to compute biodiversity impacts
def compute_land_use_metrics(use, biome_type, land_use_type, intensity, years_since_conversion):
    key = f"{land_use_type}_{intensity}"
    msa = msa_coefficient_by_land_use.get(key, None)
    biome_fraction = biome_share.get(biome_type, None)
    ess_value = biome_ess_values.get(biome_type, 0)
    restoration_value = restoration_costs.get(biome_type, 0)

    if msa is None:
        raise ValueError(f"Invalid land use type or intensity: {key}")
    if biome_fraction is None:
        raise ValueError(f"Invalid biome type: {biome_type}")

    land_ob = use * biome_fraction * msa
    land_cb = land_ob / years_since_conversion if years_since_conversion > 0 else None
    land_use_cost = round(land_ob * ess_value, 2)
    land_use_change_cost = round(land_cb * restoration_value, 2) if land_cb is not None else None

    return {
        "MSA loss coefficient": msa,
        "Biome Share": biome_fraction,
        "LAND-Ob (MSA.ha.yr/unit)": land_ob,
        "LAND-Cb (MSA.ha/unit)": land_cb,
        "Land Use Cost (€ per unit)": f"{land_use_cost:.2f}",
        "Land Use Change Cost (€ per unit)": f"{land_use_change_cost:.2f}" if land_use_change_cost is not None else "N/A"
    }

# 6. Compare food forest vs conventional agriculture EXAMPLE
food_forest = compute_land_use_metrics(
    use=0.002,
    biome_type='grassland',
    land_use_type='cropland',
    intensity='minimal',
    years_since_conversion=20
)

conventional_agriculture = compute_land_use_metrics(
    use=0.002,
    biome_type='grassland',
    land_use_type='cropland',
    intensity='intense',
    years_since_conversion=20
)

food_forest, conventional_agriculture


({'MSA loss coefficient': 0.6,
  'Biome Share': 0.68,
  'LAND-Ob (MSA.ha.yr/unit)': 0.000816,
  'LAND-Cb (MSA.ha/unit)': 4.08e-05,
  'Land Use Cost (€ per unit)': '1.98',
  'Land Use Change Cost (€ per unit)': '0.01'},
 {'MSA loss coefficient': 0.9,
  'Biome Share': 0.68,
  'LAND-Ob (MSA.ha.yr/unit)': 0.001224,
  'LAND-Cb (MSA.ha/unit)': 6.12e-05,
  'Land Use Cost (€ per unit)': '2.97',
  'Land Use Change Cost (€ per unit)': '0.02'})